In [1]:
import numpy as np
from pprint import pprint
from itertools import permutations

In [2]:
input_day08_test = ""
with open("input-day08-test") as f:
    input_day08_test = f.read().strip()

input_day08 = ""
with open("input-day08") as f:
    input_day08 = f.read().strip()

In [3]:
# 7-bit representation of each wire
REGULAR_WIRES = {
    'a': 0b0000001,  #  1
    'b': 0b0000010,  #  2
    'c': 0b0000100,  #  4
    'd': 0b0001000,  #  8
    'e': 0b0010000,  # 16
    'f': 0b0100000,  # 32
    'g': 0b1000000,  # 64
}

REGULAR_SIGNALS = {
    '0': set(list('abcefg')),
    '1': set(list('cf')),
    '2': set(list('acdeg')),
    '3': set(list('acdfg')),
    '4': set(list('bcdf')),
    '5': set(list('abdfg')),
    '6': set(list('abdefg')),
    '7': set(list('acf')),
    '8': set(list('abcdefg')),
    '9': set(list('abcdfg')),
}

In [4]:
def get_signal_bits_regular(signal_chars):
    return get_signal_bits_custom(signal_chars, REGULAR_WIRES)

def get_signal_bits_custom(signal_chars, custom_wires):
    if type(signal_chars) is str:
        signal_chars = set(list(signal_chars))
    return sum([custom_wires[x] for x in signal_chars])

assert get_signal_bits_regular('a')   == 0b0000001
assert get_signal_bits_regular('ac')  == 0b0000101
assert get_signal_bits_regular('ca')  == 0b0000101
assert get_signal_bits_regular('cgf') == 0b1100100
assert get_signal_bits_regular('gfc') == 0b1100100

In [5]:
def get_signal_chars_regular(signal_bits):
    return get_signal_chars_custom(signal_bits, REGULAR_WIRES)
    
def get_signal_chars_custom(signal_bits, custom_wires):
    bits_set = {
        'a': custom_wires['a'] & signal_bits,
        'b': custom_wires['b'] & signal_bits,
        'c': custom_wires['c'] & signal_bits,
        'd': custom_wires['d'] & signal_bits,
        'e': custom_wires['e'] & signal_bits,
        'f': custom_wires['f'] & signal_bits,
        'g': custom_wires['g'] & signal_bits,
    }
    return set([k for k, v in bits_set.items() if v])

assert get_signal_chars_regular(0b0000001) == set(['a'])
assert get_signal_chars_regular(0b0000101) == set(['a','c'])
assert get_signal_chars_regular(0b1100100) == set(['c', 'f', 'g'])

In [6]:
def get_signal_chars_for_digit_regular(digit):
    assert type(digit) is str
    return REGULAR_SIGNALS[digit]

def get_signal_chars_for_digit_custom(digit, custom_wires):
    assert type(digit) is str
    signal_chars_normal = get_signal_chars_for_digit_regular(digit)
    return encode_signal_chars(signal_chars_normal, custom_wires)

def encode_signal_chars(signal_chars_normal, custom_wires):
    if len(set(custom_wires.values())) < len(custom_wires):
        raise ValueError("duplicated bit combination in custom wire")
    result = set()
    for c in signal_chars_normal:
        c_bits = get_signal_bits_regular(c)
        for k, v in custom_wires.items():
            if c_bits == v:
                result.add(k)
                break
        else:
            raise ValueError("bit combination not found in custom wire")
    return result
        

# d changed by e
custom_wires_1 = {
    'a': 0b0000001,  #  1
    'b': 0b0000010,  #  2
    'c': 0b0000100,  #  4
    'e': 0b0001000,  #  8
    'd': 0b0010000,  # 16
    'f': 0b0100000,  # 32
    'g': 0b1000000,  # 64
}

assert get_signal_chars_for_digit_regular('0') == set(list('abcefg'))
assert get_signal_chars_for_digit_custom('0', custom_wires_1) == set(list('abcdfg'))


#def get_display_char(signal_bits):
#    for s_char, s_bits in signals.items()

In [7]:
def get_bitcount_signals(signals):
    bitcount_signals = dict(zip(range(8), [0]*8))
    for k in bitcount_signals.keys():
        bitcount_signals[k] = []
    for k, v in signals.items():
        bitcount_signals[len(v)].append(k)
    return bitcount_signals
get_bitcount_signals(REGULAR_SIGNALS)

{0: [],
 1: [],
 2: ['1'],
 3: ['7'],
 4: ['4'],
 5: ['2', '3', '5'],
 6: ['0', '6', '9'],
 7: ['8']}

In [8]:
def get_bitcount_freq(signals):
    bitcount_signals = get_bitcount_signals(signals)
    bitcount_freq = {}
    for k, v in bitcount_signals.items():
        bitcount_freq[k] = len(v)
    return bitcount_freq
get_bitcount_freq(REGULAR_SIGNALS)

{0: 0, 1: 0, 2: 1, 3: 1, 4: 1, 5: 3, 6: 3, 7: 1}

In [9]:
# counts the number of occurrences of the digits 1, 4, 7 or 8
def count_1478(input_data, signals):
    count = 0
    bitcount_freq = get_bitcount_freq(signals)
    for row in input_data.split("\n"):
        for entry in row.split("|")[1].split():
            entry_bits = get_signal_bits_regular(entry)
            entry_bitcount = bitcount_freq[entry_bits.bit_count()]
            if entry_bitcount == 1:
                count += 1
    return count

count_1478(input_day08_test, REGULAR_SIGNALS)

26

In [10]:
print("Solution to part 1:")
count_1478(input_day08, REGULAR_SIGNALS)

Solution to part 1:


521

In [11]:
# counts the number of occurrences of the digits 1, 4, 7 or 8
def decode_row(entry):
    entry_signals = entry.split("|")[0].split()
    entry_displays = entry.split("|")[1].split()
    # brute force all 5040 wire maps,
    wiremap_counter = -1
    for a, b, c, d, e, f, g in permutations([1, 2, 4, 8, 16, 32, 64], 7):
        wiremap_counter += 1
        custom_map = {'a': a, 'b': b, 'c': c, 'd': d, 'e': e, 'f': f,'g': g}
        # create the digit map for this custom wiremap
        digit_map = dict([(str(i), get_signal_chars_for_digit_custom(str(i), custom_map)) for i in range(10)])
        digit_map = dict(
            [("".join(sorted(get_signal_chars_for_digit_custom(str(i), custom_map))), str(i)) 
             for i in range(10)])
        found = True
        for entry_signal in entry_signals:
            entry_key = "".join(sorted(list(entry_signal)))
            if not entry_key in digit_map:
                break
        else:
            result = int("".join([digit_map["".join(sorted(list(ed)))] for ed in entry_displays]))
            print("Solution found on wiremap", wiremap_counter, ":", result)
            return result
    raise ValueError("solution not found for", entry)

print("---")
assert 5353 == decode_row("acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf")
assert 8394 == decode_row("be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe")
assert 9781 == decode_row("edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc")
assert 1197 == decode_row("fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg")
assert 9361 == decode_row("fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb")
assert 4873 == decode_row("aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea")
assert 8418 == decode_row("fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb")
assert 4548 == decode_row("dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe")
assert 1625 == decode_row("bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef")
assert 8717 == decode_row("egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb")
assert 4315 == decode_row("gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce")

---
Solution found on wiremap 2016 : 5353
Solution found on wiremap 3171 : 8394
Solution found on wiremap 2924 : 9781
Solution found on wiremap 2204 : 1197
Solution found on wiremap 2476 : 9361
Solution found on wiremap 4334 : 4873
Solution found on wiremap 4022 : 8418
Solution found on wiremap 3553 : 4548
Solution found on wiremap 3248 : 1625
Solution found on wiremap 2905 : 8717
Solution found on wiremap 2772 : 4315


In [13]:
def decode_input(input_rows):
    results = []
    for row in input_rows.split("\n"):
        results.append(decode_row(row))
    return sum(results)

In [14]:
# Part 1
result_part1 = decode_input(input_day08_test)
print("Result for part 1:", result_part1)
assert 61229 == result_part1

Solution found on wiremap 3171 : 8394
Solution found on wiremap 2924 : 9781
Solution found on wiremap 2204 : 1197
Solution found on wiremap 2476 : 9361
Solution found on wiremap 4334 : 4873
Solution found on wiremap 4022 : 8418
Solution found on wiremap 3553 : 4548
Solution found on wiremap 3248 : 1625
Solution found on wiremap 2905 : 8717
Solution found on wiremap 2772 : 4315
Result for part 1: 61229


In [15]:
# Part 2
result_part2 = decode_input(input_day08)
print("-----")
print("Result for part 2:", result_part2)


Solution found on wiremap 4462 : 5311
Solution found on wiremap 2412 : 1730
Solution found on wiremap 194 : 8687
Solution found on wiremap 3921 : 4842
Solution found on wiremap 3923 : 5158
Solution found on wiremap 2058 : 4106
Solution found on wiremap 1130 : 8180
Solution found on wiremap 1120 : 7011
Solution found on wiremap 4624 : 7823
Solution found on wiremap 3995 : 1017
Solution found on wiremap 2175 : 7755
Solution found on wiremap 3879 : 1741
Solution found on wiremap 2212 : 9878
Solution found on wiremap 3839 : 1081
Solution found on wiremap 465 : 909
Solution found on wiremap 3253 : 5010
Solution found on wiremap 2307 : 888
Solution found on wiremap 1141 : 1729
Solution found on wiremap 1661 : 1145
Solution found on wiremap 4034 : 2861
Solution found on wiremap 2940 : 6723
Solution found on wiremap 604 : 8887
Solution found on wiremap 1126 : 1888
Solution found on wiremap 214 : 1674
Solution found on wiremap 3477 : 6147
Solution found on wiremap 1060 : 8378
Solution found on 